In [1]:
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA
import numpy as np
import cdd

v1 = np.array([[0.0, 0.0,0.0,0.0],[1.0, 1.0,1.0,1.0], [1.0/2, 1.0/2,1.0/2,1.0/2], [1.0/4, 1.0/4,1.0/4,1.0/4]])
v2 = np.array([[0.0, 0.0,0.0,0.0],[1.0, 1.0,1.0,1.0]])
v3 = np.array([[1.0, 1.0,0.0,1.0],[1.0, 0.0,2.0,1.0]])
v4 = np.array([[1.0,0.0,1.0],[ 0.0,2.0,1.0], [ 0.0,0.0,1.0], [ 1.0,2.0,1.0]])


print(np.linalg.matrix_rank(v1))
print(np.linalg.matrix_rank(v2))
print(np.linalg.matrix_rank(v3))
print(np.linalg.matrix_rank(v4))


1
1
2
3


In [3]:
print (np.linalg.svd(v1))

(array([[-2.22044605e-16,  1.00000000e+00,  1.38777878e-17,
         5.55111512e-17],
       [-8.72871561e-01,  1.11022302e-16, -2.18217890e-01,
        -4.36435780e-01],
       [-4.36435780e-01,  0.00000000e+00, -5.08513756e-02,
         8.98297249e-01],
       [-2.18217890e-01,  0.00000000e+00,  9.74574312e-01,
        -5.08513756e-02]]), array([2.29128785e+00, 2.20303052e-16, 0.00000000e+00, 0.00000000e+00]), array([[-5.00000000e-01, -5.00000000e-01, -5.00000000e-01,
        -5.00000000e-01],
       [-8.66025404e-01,  2.88675135e-01,  2.88675135e-01,
         2.88675135e-01],
       [ 0.00000000e+00, -8.75605293e-17, -7.07106781e-01,
         7.07106781e-01],
       [ 0.00000000e+00,  8.16496581e-01, -4.08248290e-01,
        -4.08248290e-01]]))


In [5]:
print (np.linalg.svd(v2))

(array([[ 0., -1.],
       [-1.,  0.]]), array([2., 0.]), array([[-0.5       , -0.5       , -0.5       , -0.5       ],
       [ 0.8660254 , -0.28867513, -0.28867513, -0.28867513],
       [ 0.        , -0.57735027,  0.78867513, -0.21132487],
       [ 0.        , -0.57735027, -0.21132487,  0.78867513]]))


In [6]:
print (np.linalg.svd(v3))

(array([[-0.4472136 , -0.89442719],
       [-0.89442719,  0.4472136 ]]), array([2.64575131, 1.41421356]), array([[-0.50709255, -0.16903085, -0.6761234 , -0.50709255],
       [-0.31622777, -0.63245553,  0.63245553, -0.31622777],
       [-0.49621768,  0.74419182,  0.37209591, -0.24797415],
       [-0.62978184, -0.13269175, -0.06634587,  0.76247358]]))


In [7]:
print (np.linalg.svd(v4))

(array([[-2.39614662e-01,  8.16496581e-01,  1.60991139e-01,
        -5.00000000e-01],
       [-6.33960807e-01, -4.08248290e-01, -4.25942518e-01,
        -5.00000000e-01],
       [-1.54731482e-01,  4.08248290e-01, -7.47924797e-01,
         5.00000000e-01],
       [-7.18843987e-01, -2.22044605e-16,  4.82973418e-01,
         5.00000000e-01]]), array([3.36028312, 1.41421356, 0.84172286]), array([[-0.28523152, -0.8051731 , -0.51994159],
       [ 0.57735027, -0.57735027,  0.57735027],
       [ 0.76505532,  0.13550992, -0.6295454 ]]))


In [ ]:
v_min_size = min(v.shape)
dim = v.shape[1]
pca = PCA(n_components=v_min_size)
pca.fit(v)
C = pca.components_
mean = pca.mean_
low_dim_pts = pca.transform(v)
print (v)
print (C)
print (mean)
print (low_dim_pts)
print (np.matmul(C, C.T))

In [ ]:
print (np.matmul(C.T,low_dim_pts.T))

In [ ]:
explain_ratio = np.cumsum(pca.explained_variance_ratio_)

first_dims = np.argmax(explain_ratio >= 1.0-0.00001)

low_dim_pts_reduct = low_dim_pts[:,0:first_dims+1]

# decide how many to remove...
print (low_dim_pts_reduct)

In [ ]:
if low_dim_pts_reduct.shape[1] == 1:
    lb = np.min(low_dim_pts_reduct)
    ub = np.max(low_dim_pts_reduct)
    # x <= ub  -x >= -lb
    A = np.array([[1.0],[-1.0]])
    b = np.array([ub, -lb])
else:
    # convex hull will get you Ax + b <= 0
    hull = ConvexHull(points=low_dim_pts_reduct)
    A = hull.equations[:,:-1]
    b = -hull.equations[:,-1]
print (A,b)

In [ ]:
n_rows = A.shape[0]
n_cols = A.shape[1]

num_var_to_add = v_min_size - (first_dims+1)

zero4 = np.zeros((n_rows,num_var_to_add))

rows_to_append = []
for idx in range(num_var_to_add):
    l = [0.0]*num_var_to_add
    l[idx] = 1.0
    rows_to_append.append(np.append(np.zeros((1,n_cols)), l))
    l[idx] = -1.0
    rows_to_append.append(np.append(np.zeros((1,n_cols)), l))

trueA = np.vstack([np.hstack([A, zero4])] + rows_to_append)
trueB = np.append(b, [0.0, 0.0]*num_var_to_add)
print (trueA)
print (trueB)

In [ ]:
print (mean)
print (trueA.shape)
print (trueA)
print (C)
print (C.shape)
print (trueB)

In [ ]:
finalA = np.matmul(trueA, C)
finalA

In [ ]:
np.matmul(finalA, mean)

In [ ]:
np.matmul(finalA, mean)+trueB

In [ ]:
0.8660254/0.28867513

In [ ]:
1.78885438/0.4472136

In [ ]:
0.3527864/0.4472136